In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name

In [2]:
key = pd.read_csv('key.csv')
weather = pd.read_csv('weather.csv', parse_dates=['date'])
train = pd.read_csv('train.csv', parse_dates=['date'])

In [3]:
train = pd.merge(train,key)

In [4]:
rt = train[train['units'] > 0].sort_values(by=['units'], ascending=False).reset_index(drop=True)

In [5]:
# weahter: codesum split & get dummies

# extract codsum unique values
cs_iter = (set(x.split(' ')) for x in weather.codesum)
cs = sorted(set.union(*cs_iter))

dummies = pd.DataFrame(np.zeros((len(weather), len(cs))), columns = cs)

for i, gen in enumerate(weather.codesum):
    dummies.ix[i, gen.split(' ')] = 1
    
weather_cs = weather.join(dummies)
del weather_cs['codesum']
del weather_cs['']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [6]:
df = pd.merge(rt, weather_cs, on=['date', 'station_nbr'])

In [7]:
hol = pd.read_table('holiday_names.txt', sep='\s+', names = ['year', 'month', 'day', 'name'])
hol.tail()

,year,month,day,name
53,2014,Jun,15,FathersDay
54,2014,Jul,4,IndependenceDay
55,2014,Sep,1,LaborDay
56,2014,Oct,13,ColumbusDay
57,2014,Oct,31,Halloween


In [8]:
hol.month.unique()

array(['Jan', 'Feb', 'Apr', 'May', 'Jun', 'Jul', 'Sep', 'Oct', 'Nov',
       'Dec', 'Mar'], dtype=object)

In [9]:
dic = {'Jan': 1,
       'Feb': 2,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12,
        'Mar': 5}

hol['monthnumber'] = hol.month.map(dic)

hol.columns = ['year', 'months', 'day', 'name', 'month']


hol = hol.assign(Date = pd.to_datetime(hol[['year', 'month', 'day']]))
hol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
year      58 non-null int64
months    58 non-null object
day       58 non-null int64
name      58 non-null object
month     58 non-null int64
Date      58 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 2.8+ KB


In [10]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df[['date']].apply(lambda x: dt.datetime.strftime(x['date'], '%a'), axis=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118696 entries, 0 to 118695
Data columns (total 54 columns):
date           118696 non-null datetime64[ns]
store_nbr      118696 non-null int64
item_nbr       118696 non-null int64
units          118696 non-null int64
station_nbr    118696 non-null int64
tmax           118696 non-null object
tmin           118696 non-null object
tavg           118696 non-null object
depart         118696 non-null object
dewpoint       118696 non-null object
wetbulb        118696 non-null object
heat           118696 non-null object
cool           118696 non-null object
sunrise        118696 non-null object
sunset         118696 non-null object
snowfall       118696 non-null object
preciptotal    118696 non-null object
stnpressure    118696 non-null object
sealevel       118696 non-null object
resultspeed    118696 non-null object
resultdir      118696 non-null object
avgspeed       118696 non-null object
BCFG           118696 non-null float64
BLDU      

In [ ]:
# counting M

columns = list(df.columns)

def count_M(columns):
    ls_M_counting = []
    ls_M_columns= []
    ls_all_counting = []
    
    for i in columns:
        a = df[i].value_counts()
        b = df[i].name
        
        if 'M' in a:
            ls_M_counting.append(a['M'])
            ls_M_columns.append(b)
            ls_all_counting.append(a.sum())
    return ls_M_counting, ls_M_columns, ls_all_counting

        
    
k = count_M(columns)
k = np.array(k).T

M_columns = ['M', 'column', 'total']
dfdf = pd.DataFrame(data = k, columns = M_columns)
dfdf['M'] = dfdf['M'].astype(int)
dfdf['total'] = dfdf['total'].astype(int)
dfdf['Pct'] = dfdf['M'] / dfdf['total'] * 100
dfdf.sort_values(by='Pct', ascending=False)

In [ ]:
for col in df.columns:
    

In [ ]:
# M -> numeric

for col in df.columns:
    if df[col] == 'weekday': continue
    elif df[col].dtypes == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else: continue
